# TensorFlow Playground
Welcome to the TensorFlow Playground! This notebook is modeled after the official [playground.tensorflow.org](http://playground.tensorflow.org) web application which offers an playful way to test different feed forward neural network configurations. This notebook gives you the opportunity to actually implement different network configurations you might already have tested in the web app.

## Preperations
You can basically ignore this first code cell below. All it does is generate the four different datasets and provide a visualization function. What you can do here is adjust N, the amount of samples we generate. If you feel fancy you can also tweak the datasets to make the problems harder by for example adding some noise to the spiral.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

import numpy as np
from numpy.random import normal, random_sample, permutation
from numpy.linalg import norm

def viz(data, labels, progress, accuracy):
    """Function visualizing training progress."""
    if not hasattr(fig, 'viz'): viz.fig = plt.figure('Result')
    viz.fig.suptitle("{:3.2%} done with accuracy {:2.2f}"
                     .format((epoch+1)/epochs, accuracy))
    plt.scatter(*zip(*data), c=labels, cmap='bwr')
    viz.fig.canvas.draw()


class Datasets:
    """Provides dataset generators and helper functions."""

    # Returns n floats uniformly distributed between a and b.
    uniform = lambda self, a, b, n=1: (b - a) * random_sample((1, n)) + a
    
    def split(self, sets, split=10):
        """Splits samples into train and test sets and reshapes target labels as required."""
        n = sets[0].shape[0]
        permut = permutation(n).astype(int)
        return tuple(zip(*((d[permut[n//split:]], d[permut[:n//split]]) for d in sets)))

    def circle(self, n=1000, radius=5, noise=0):
        """Generates data with one class describing an inner and one an outer circle."""
        radius = 5
        r = np.append(self.uniform(0, radius*.5, n//2), 
                      self.uniform(radius*.7, radius, n//2))
        angle = self.uniform(0, 2*np.pi, n)
        x = r * np.sin(angle) + self.uniform(-radius,radius,n) * noise
        y = r * np.cos(angle) + self.uniform(-radius,radius,n) * noise
        xy = np.vstack((x,y))
        t = np.less(norm(xy, axis=0), radius*.5)
        return (xy.T, t[:, np.newaxis])
    
    def xor(self, n=1000, padding=.3, noise=0):
        """Generates xor data."""
        x = self.uniform(-5, 5, n); x[x>0] += padding; x[x<0] -= padding
        y = self.uniform(-5, 5, n); y[y>0] += padding; y[y<0] -= padding
        x += self.uniform(-5, 5, n) * noise
        y += self.uniform(-5, 5, n) * noise
        t = np.less(x*y, 0).flatten()
        return (np.vstack((x, y)).T, t[:, np.newaxis])

    def gauss(self, n=1000, noise=0):
        """Generates two blops of gauss data, each describing a class."""
        x = normal( 2, 1+noise, (n//2, 2))
        y = normal(-2, 1+noise, (n//2, 2))
        t = np.append(np.ones(n//2), np.zeros(n//2))
        return (np.vstack((x, y)), t[:, np.newaxis])

    def genSpiral(self, delta, n, noise=0):
        """Generates a spiral of n points with rotation-offset delta."""
        points = np.arange(n)
        r = points / n * 5
        t = 1.75 * points / n * 2 * np.pi + delta
        x = r * np.sin(t) + self.uniform(-1, 1, n) * noise
        y = r * np.cos(t) + self.uniform(-1, 1, n) * noise
        return np.vstack((x, y)).T
    
    def spiral(self, n=1000, noise=0):
        """Generates a dataset of two intertwined spirals."""
        a = self.genSpiral(    0, n//2, noise)
        b = self.genSpiral(np.pi, n//2, noise)
        t = np.append(np.ones(n//2), np.zeros(n//2))
        return (np.vstack((a, b)), t[:, np.newaxis])

datasets = Datasets()

fig = plt.figure('Available Datasets')
circle = datasets.circle(200); ax1 = plt.subplot(221); ax1.set_title('circle')
ax1.scatter(*zip(*circle[0]), c=circle[1], cmap='bwr')
xor = datasets.xor(200); ax2 = plt.subplot(222); ax2.set_title('xor')
ax2.scatter(*zip(*xor[0]), c=xor[1], cmap='bwr')
gauss = datasets.gauss(200); ax3 = plt.subplot(223); ax3.set_title('gauss')
ax3.scatter(*zip(*gauss[0]), c=gauss[1], cmap='bwr')
spiral = datasets.spiral(100); ax4 = plt.subplot(224); ax4.set_title('spiral')
ax4.scatter(*zip(*spiral[0]), c=spiral[1], cmap='bwr')
del circle, xor, gauss, spiral

## Data
In the following cell we generate our dataset. Run the cell above for visualizations of the different choices. To use anything but the gauss dataset you can simply exchange the `datasets.gauss` function call with either `datasets.circle`, `datasets.xor` or `datasets.spiral`. Additionally you can add some noise to the data to make solving the problem more difficult. Some information about the provided data structures:

The two `*_data` variables contain the actual data in 2 dimensional numpy arrays, the first axis describes individual observations, the second axis the `x` and `y` values. Therefore they have a shape of `(m, 2)`. Similarilly the two `*_target` variables are arrays containing the corresponding labels, again in a shape of `(m, 2`), $samples \times classes$: The second axis binarily describes weather a sample belongs to the first class (the first value in the second axis will equal `1`, the second value `0`) or to the second class (second value will be `1`..).

As the names suggest, `train_data` and `train_targets` are designated for training and contain a lot more data (by default $9/10$ of the data is training data) and `test_data` and `test_targets` are meant for evaluating the networks performance. You can change the split between training and testing data by adjusting the second parameter for the `datasets.split` function below.

In [ ]:
# datasets.circle(), datasets.xor(), datasets.gauss(), datasets.spiral()
data, labels = datasets.gauss(n=2000, noise=.5)
(train_data, train_labels), (test_data, test_labels) = datasets.split((data, labels), 10)

plt.figure('Choosen dataset')
plt.scatter(*zip(*data), c=labels, cmap='bwr')
fig.canvas.draw()

## It's your turn!
In the next cell stuff gets interesting. We preimplemented a basic model which is sufficiently powerful for the gauss dataset. It consists of 2 neurons in the input layer to feed `x` and `y` values into, 4 neurons in the single hidden layer and again 2 neurons in the output layer for giving probabilities for each of our two classes.

Instead of using buttons and dropdowns you will need to get your hands dirty here in order to being able to cope with more complex datasets! While we provide some guidance to mock the experience of the web application it will be also super useful to checkout the [TensorFlow API documentation](https://www.tensorflow.org/versions/r0.9/api_docs/index.html). Because the documentation does not give any theoretical background on all those different functions you might also want to regularily consultate Wikipedia.

  * Adjust the hidden layer, play around with more or even fewer neurons.
  * Add another hidden layer! You will need to adjust the matrix multiplications for its surrounding layers in order to integrate it.
  * Change the activation functions. By default both, the hidden and the output layer, use the sigmoid activation function. Similar to the web application you can use `relu` or `tanh`. You might also consider trying to implement your own `linear` activation function -- TensorFlow does not provide one by itself.
  * Take a look into different error measures and optimizers. This is nothing you can change in the playground web application but actually a powerful tool to tweak your networks performance through better training. Checkout the [optimizers chapter](https://www.tensorflow.org/versions/r0.9/api_docs/python/train.html#optimizers) in the documentation.
  * Finally take a look at the actual training loop. Tweak the amount of iterations and change the batch size. You can actually also tweak the batch size in the web app, take a look at the bottom left. 

When you solved all the datasets (choose in the code cell above) you can also start adding noise to the data or take less data for training by adjusting the test/train split (again, both definable above).

In [ ]:
import tensorflow as tf

x = tf.placeholder(tf.float32, shape=[None, 2], name='input_layer')

W_h = tf.Variable(tf.random_normal([2,4]), name='hidden_weights')
b_h = tf.Variable(tf.zeros([4]), name='hidden_bias')
h = tf.nn.sigmoid(tf.matmul(x,W_h) + b_h, name='hidden_layer')

W_y = tf.Variable(tf.random_normal([4,1]), name='output_weights')
b_y = tf.Variable(tf.zeros([1]), name='output_bias')
y = tf.nn.sigmoid(tf.matmul(h,W_y) + b_y, name='output_layer')

t = tf.placeholder(tf.float32, shape=[None, 1], name='targets')

error = tf.reduce_mean(tf.squared_difference(t,y), name='mean_squared_error')
optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(error, name='gradient_descent')

prediction_check = tf.equal(tf.round(y), t, name='prediction_check')
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32), name='accuracy_measure')

sess = tf.Session()
sess.run(tf.initialize_all_variables())

epochs = 1000
for epoch in range(epochs):
    split = permutation(len(train_labels))
    batch_labels = train_labels[split[:10]]
    batch_data = train_data[split[:10]]
    sess.run(optimizer, {x: batch_data, t: batch_labels})
    if (epoch+1)%100 == 0:
        perf = sess.run(accuracy, {x: test_data, t: test_labels})
        predictions = sess.run(y, {x: data})
        viz(data, predictions, (epoch+1)/epochs, perf)

perf = sess.run(accuracy, {x: test_data, t: test_labels})
predictions = sess.run(y, {x: data})
viz(data, predictions, 1, perf)